# Introduction to CycleGANs

In this notebook will look at understanding the key research paper on cycleGANs. 

I have then included two further notebooks in this folder. Both contain a tensorflow implementation of this paper. The original can be found here:

https://github.com/xhujoy/CycleGAN-tensorflow

One of the notebooks contains the implementation with space for you to comment all the code sections. This is really important excersize, particularly if you are not too familiar with tensorflow. The other notebook contains all of the comments I made when trying to understand the code, so refer to this if you get stuck. It took me 3/4 sessions before I felt like I understood what this code was doing, so don't feel like you need to understand it all in one go.

We will not try and implement our own version of a cycleGAN from scratch but we should learn enough from this commenting excersie to understand the key components for implementation of cariGAN. Once you have finished the commenting excersie run the algorithm, locally or on GoogleColab (it took me 24 hours to train in colab, so I'd suggest doing it there unless you have a powerful setup.)

### Resources:

The key research paper on GANs (which can be found in this folder) is very famous and there are many good articles whcih summarise. I suggest reading/watching the following resources which neatly summarise this paper:

https://medium.com/coding-blocks/introduction-to-cyclegans-1dbdb8fbe781

https://towardsdatascience.com/cyclegan-learning-to-translate-images-without-paired-training-data-5b4e93862c8d

https://medium.com/@jonathan_hui/gan-cyclegan-6a50e7600d7

https://www.youtube.com/watch?v=MgdAe-T8obE

Finally give the full paper a read through, don't worry if you don't understand all the terminology, just the high level concepts need to be understood.

https://arxiv.org/pdf/1703.10593.pdf

### Cycle GANs Introduction

After seeing the horse2zebra gif above, most of you would be thinking of a following approach : Prepare a dataset of Horses and Zebras in the same environment, in exactly the same locations and then create some kind of a mapping between the two with the help of a Neural Network. But that’s not how it works because it would be close to impossible to get such a dataset. The beauty of the algorithm lies in achieving the same result in a smart and easy way with a dataset containing just the images of Horses and Zebras.

Cycle GANs aim to capture special characteristics of one image collection and figuring out how these characteristics could be translated into the other image collection, all in the absence of any paired training examples.

### CycleGAN Definitions:

$x_{i} ∈ X$: example in source set e.g. a photograph

$y_{j} ∈ Y$: example in target set e.g. a Van Gough painting

<b>NOTE: </b>$x_{i} and y_{j}$ are unpaired.

$G: X \rightarrow Y$: A generator G to convert a photograph to a Van Gogh style picture.

$F: Y \rightarrow X$: A generator F to convert a Van Gogh style picture to a photograph.

$\hat{X}$: set of generated photographs

$\hat{Y}$: set of generated Van Gough

$D_{y}$: A discriminator D to identify generated target images (generated Van Gough) from target images (real Van Gough)

$D_{x}$: Reverse of $D_{y}$, we just reverse the data flow and build an additional discriminator. Identify generated source images (generated photograph) from source images (real photograph)

$\lambda$: cycle loss parameter (controls the relative importance of the loss functions)

<img src="notebook_images/network_arcitecture_A.png">
<img src="notebook_images/network_arcitecture_B.png">

### Definitions:

###### Convolutional Layer

Powerful visual layer

###### Residual Block (Convolutional)

A residual block is simply when the activation of a layer is fast-forwarded to a deeper layer in the neural network. This fas-forward connection is called a skip connection.Very deep neural networks are hard to train as they are more prone to vanishing or exploding gradients. A residual block helps solve this problem.

###### Transpose Convolutional Layer

Simply the transpose of the convolutional layer (above)

### Generator Architecture

<img src="notebook_images/generator_arcitecture.png">



## Discriminator Definitions:
###### Convolutional Layer

Powerful visual layer

###### Instance Normalisation (Contrast Normaisation)

Where batch norm computes one mean and std dev (thus making the distribution of the whole layer Gaussian), instance norm makes each individual image distribution look Gaussian, but not jointly.

Output stylised images should be not depend on the contrast of the input content image and hence Instance normalisation helps

https://arxiv.org/pdf/1607.08022.pdf

###### Leaky ReLU:

https://www.quora.com/What-are-the-advantages-of-using-Leaky-Rectified-Linear-Units-Leaky-ReLU-over-normal-ReLU-in-deep-learning

With a Leaky ReLU (LReLU), you won’t face the “dead ReLU” problem which happens when your ReLU always have values under 0 - this completely blocks learning in the ReLU because of gradients of 0 in the negative part. So:

ReLU: The derivative of the ReLU is 1 in the positive part, and 0 in the negative part.
LReLU: The derivative of the LReLU is 1 in the positive part, and is a small fraction in the negative part.
Now, think about the chain rule in the backward pass. If the derivative of the slope of the ReLU is of 0, absolutely no learning is performed on the layers below the dead ReLU, because 0 will be multiplied to the accumulated gradient for the weight update. Thus, you can have dead neurons. This problem doesn’t happen with LReLU or ELU for example, they will always have a little slope to allow the gradients to flow on.

An advantage of using a LReLU is thus that you can worry less about the initialization of your neural network. In the ReLU case, you can end up with a neural network that never learns if the neurons are not activated at the start. You may have lots of dead ReLU without even knowing. However, ReLU computes faster and can introduce a sort of “optimal brain damage” regularization in your machine learning algorithm.

###### Sigmoid

Used in final layer as a probability layer. Remember the discriminator network determines if the generator has done it's job properly or not.

### Discriminator Architecture

<img src="notebook_images/discriminator_arcitecture.png">

### Loss Function // Objective Functions

##### Adversarial Loss

Our discriminator networks aim to assess if the output from our generator networks qualify. That is to say, $D_{y}(G(x_{i})) \rightarrow 1$ as the output of $G$ becomes more and more accurate. Similarly $D_{x}(F(y_{i})) \rightarrow 1$ as $F$ gets more accurate.

We apply adversarial losses to both mapping functions. For the mapping function G : X → Y and its discriminator DY. G tries to generate images G(x) that look similar to images from domain Y , while DY aims to distinguish between translated samples G(x) and real samples y. G aims to minimize this objective against an adversary D that tries to maximize it, i.e., minG maxDY LGAN(G, DY , X, Y). We introduce a similar adversarial loss for the mapping
function F : Y → X and its discriminator DX as well. 

$ Loss_{adv}(G, D_{y}, X) = 1/m \cdot \sum_{i=1}^m (1 - D_{y}(G(x_{i})))^2 $ 

$ Loss_{adv}(F, D_{x}, Y) = 1/m \cdot \sum_{i=1}^m (1 - D_{x}(F(y_{i})))^2 $

##### Cycle Consistancy Loss

Adversarial loss alone will not (constrain) the network sufficiently to produce the required results. Any network (with large enough capacity) should be able to learn a mapping form any set of input images to any random permutations of output images. We therefore need to reduce the space of possible mapping functions. We do this by adding cycle consistancy loss to our objective function. Cycle consistancy loss simply constains the mappings so that we can get back to our original input. That is to say $x \rightarrow G(x) \rightarrow F(G(x)) \approx x$

$ Loss_{cyc}(G, F, X, Y) = 1/m \cdot \sum_{i=1}^m [[F(G(x_{i})) - x_{i}] + [G(F(y_{i})) - y_{i}]] $

##### Total Loss

$ Loss_{full} = Loss_{adv} + \lambda \cdot Loss_{cyc} $

$ Loss_{full} = Loss_{adv}(G, D_{y}, X, Y) + Loss_{adv}(F, D_{x}, Y, X)+ \lambda \cdot Loss_{cyc}(G, F, X, Y)$


### Reducing Model Oscillation (Experience Replay):

In addition, CycleGAN retains a history of the last 50 generated images to train the discriminator. Discriminators and generators can overfit themselves and fall into a cycle of greedy optimization that leads to the mode collapse. We use experience replay so the discriminator will not be overzealous in beating the current generator. It needs to beat the last 50 generators to create a more generalized solution.

https://arxiv.org/pdf/1612.07828.pdf